# DRP Processing Flags

For DESC-SSim call June 29 2017

This notebook makes use of some of the LSST stack packages (i.e., `daf`). An example kernel that loads these is available in [lsst-kernel.sh](./lsst-kernel.sh).

Validated by [@kadrlica](https://github.com/kadrlica): 2017-12-06 

In [1]:
import lsst.daf.persistence as dafPersist

butler = dafPersist.Butler('/global/cscratch1/sd/descdm/DC1/rerun/DC1-imsim-dithered')
srcCat = butler.get('deepCoadd_meas', filter='r', tract=0, patch='7,9')
srcCat.getSchema()

Schema(
    (Field['L'](name="id", doc="unique ID"), Key<L>(offset=0, nElements=1)),
    (Field['Angle'](name="coord_ra", doc="position in ra/dec"), Key<Angle>(offset=8, nElements=1)),
    (Field['Angle'](name="coord_dec", doc="position in ra/dec"), Key<Angle>(offset=16, nElements=1)),
    (Field['L'](name="parent", doc="unique ID of parent source"), Key<L>(offset=24, nElements=1)),
    (Field['Flag'](name="flags_negative", doc="set if source was detected as significantly negative"), Key['Flag'](offset=32, bit=0)),
    (Field['Flag'](name="merge_footprint_i", doc="Detection footprint overlapped with a detection from filter i"), Key['Flag'](offset=32, bit=1)),
    (Field['Flag'](name="merge_footprint_r", doc="Detection footprint overlapped with a detection from filter r"), Key['Flag'](offset=32, bit=2)),
    (Field['Flag'](name="merge_footprint_z", doc="Detection footprint overlapped with a detection from filter z"), Key['Flag'](offset=32, bit=3)),
    (Field['Flag'](name="merge_footpri

More detail about the flags seen in this schema:

## 1) Primary Flag

The most important flag is the `detect_isPrimary` flag. Any kind analysis should be done only for sources `where detect_isPrimary` which will filter duplicate sources from your catalog.  

There are two types of duplicates in the source catalogs. 
1. *Deblender* parents vs. their children
2. Sources measured in the *patch and tract overlap* regions. "detect_isPrimary=True" will ensure that you have ONLY the children sources when a footprint was deblended and only sources in the inner patch and inner tract. 

### Deblender:


We refer to a group of blended objects as a “family.” A “child” refers to a particular deblended object within a family. A “parent” is also created for each family, representing the alternate hypothesis that the blend is actually a single object. For a blend, both the parent and children are entered into the catalog, with the parent having `detect_isPrimary = False`, and `deblend_nChild = <number of children it has>`. The children will have `detect_isPrimary = True`, `deblend_nChild = 0` and `parent = <id of the parent>`.  You can self join the on ID of the parent to filter out sources with exceptionally bright parents which have pathological photometry. You can also only select children by limiting to `deblend_nChild == 0`.


### Tract and Patch Overlaps:

The patches overlap to allow measurement of sources on the edges of patches. Patches have an inner BBox and an outer BBox. If the centroid falls in the innerBBox is it marked `detect_isPatchInner`. 


    (Field['Flag'](name="detect_isPatchInner", doc="true if source is in the inner region of a coadd patch"), Key['Flag'](offset=544, bit=62)),
    (Field['Flag'](name="detect_isTractInner", doc="true if source is in the inner region of a coadd tract"), Key['Flag'](offset=544, bit=63)),
    (Field['I'](name="deblend_nChild", doc="Number of children this object has (defaults to 0)"), Key<I>(offset=40, nElements=1)),
    (Field['Flag'](name="detect_isPrimary", doc="true if source has no children and is in the inner region of a coadd patch and is in the inner region of a coadd tract and is not "detected" in a pseudo-filter (see config.pseudoFilterList)"), Key['Flag'](offset=1168, bit=0)),
        
        
   
## Summary:
    
    detect_isPrimary = deblend_nChild == 0 and detect_isPatchInner and detect_isTractInner

 

In [3]:
srcCat.get("deblend_nChild")
srcCat.get("detect_isPrimary")
srcCat.get("parent")

# Limit to primary like:
idx = srcCat.get("detect_isPrimary") 
print(len(srcCat.get("base_PsfFlux_flux")[idx]))

12175


## 2) Pixel Flags

For clean samples, you may also want to filter sources w/ bad pixels. We often filter out `base_PixelFlags_flag_edge`, `base_PixelFlags_flag_saturated`, `base_PixelFlags_flag_saturatedCenter`. Using `base_PixelFlags_flag` will filter out all of the flags that start with `base_PixelFlags_flag*`.

    (Field['Flag'](name="base_PixelFlags_flag", doc="general failure flag, set if anything went wring"), Key['Flag'](offset=544, bit=22)),
    (Field['Flag'](name="base_PixelFlags_flag_offimage", doc="Source center is off image"), Key['Flag'](offset=544, bit=23)),
    (Field['Flag'](name="base_PixelFlags_flag_edge", doc="Source is outside usable exposure region (masked EDGE or NO_DATA)"), Key['Flag'](offset=544, bit=24)),
    (Field['Flag'](name="base_PixelFlags_flag_interpolated", doc="Interpolated pixel in the Source footprint"), Key['Flag'](offset=544, bit=25)),
    (Field['Flag'](name="base_PixelFlags_flag_saturated", doc="Saturated pixel in the Source footprint"), Key['Flag'](offset=544, bit=26)),
    (Field['Flag'](name="base_PixelFlags_flag_cr", doc="Cosmic ray in the Source footprint"), Key['Flag'](offset=544, bit=27)),
    (Field['Flag'](name="base_PixelFlags_flag_bad", doc="Bad pixel in the Source footprint"), Key['Flag'](offset=544, bit=28)),
    (Field['Flag'](name="base_PixelFlags_flag_suspect", doc="Source''s footprint includes suspect pixels"), Key['Flag'](offset=544, bit=29)),
    (Field['Flag'](name="base_PixelFlags_flag_interpolatedCenter", doc="Interpolated pixel in the Source center"), Key['Flag'](offset=544, bit=30)),
    (Field['Flag'](name="base_PixelFlags_flag_saturatedCenter", doc="Saturated pixel in the Source center"), Key['Flag'](offset=544, bit=31)),
    (Field['Flag'](name="base_PixelFlags_flag_crCenter", doc="Cosmic ray in the Source center"), Key['Flag'](offset=544, bit=32)),
    (Field['Flag'](name="base_PixelFlags_flag_suspectCenter", doc="Source''s center is close to suspect pixels"), Key['Flag'](offset=544, bit=33)),

## 3) Measurement Flags

Measurement algorithms have their own flags to record exceptions and abnormalities detected in the measurement. Every measurement algorithm will have a unique series of flags to record the unique failure modes and a general flag which is the `or` of the individual flags. For example:

    (Field['D'](name="base_PsfFlux_flux", doc="flux derived from linear least-squares fit of PSF model", units="count"), Key<D>(offset=696, nElements=1)),
    (Field['D'](name="base_PsfFlux_fluxSigma", doc="1-sigma flux uncertainty", units="count"), Key<D>(offset=704, nElements=1)),
    (Field['Flag'](name="base_PsfFlux_flag", doc="general failure flag"), Key['Flag'](offset=544, bit=34)),
    (Field['Flag'](name="base_PsfFlux_flag_noGoodPixels", doc="not enough non-rejected pixels in data to attempt the fit"), Key['Flag'](offset=544, bit=35)),
    (Field['Flag'](name="base_PsfFlux_flag_edge", doc="object was too close to the edge of the image to use the full PSF model"), Key['Flag'](offset=544, bit=36)),


    base_PsfFlux_flag = base_PsfFlux_flag_noGoodPixels or base_PsfFlux_flag_edge

In [4]:
srcCat.get("base_PsfFlux_flag")

array([ True,  True, False, ...,  True,  True,  True], dtype=bool)

In [5]:
# DM *DOES NOT* endorse this way of working with catalogs...

import pandas as pd

df = pd.DataFrame()
for col in srcCat.getSchema():
    name = col.field.getName()
    df[name] = srcCat.get(name)

df.to_csv('filename.csv',index=False)

# Print the head of the file
!head -n2 filename.csv

id,coord_ra,coord_dec,parent,flags_negative,merge_footprint_i,merge_footprint_r,merge_footprint_z,merge_footprint_g,merge_footprint_y,merge_footprint_u,merge_footprint_sky,merge_peak_i,merge_peak_r,merge_peak_z,merge_peak_g,merge_peak_y,merge_peak_u,merge_peak_sky,deblend_nChild,deblend_deblendedAsPsf,deblend_psfCenter_x,deblend_psfCenter_y,deblend_psfFlux,deblend_tooManyPeaks,deblend_parentTooBig,deblend_masked,deblend_skipped,deblend_rampedTemplate,deblend_patchedTemplate,deblend_hasStrayFlux,base_GaussianCentroid_x,base_GaussianCentroid_y,base_GaussianCentroid_flag,base_GaussianCentroid_flag_noPeak,base_GaussianCentroid_flag_resetToPeak,base_NaiveCentroid_x,base_NaiveCentroid_y,base_NaiveCentroid_flag,base_NaiveCentroid_flag_noCounts,base_NaiveCentroid_flag_edge,base_NaiveCentroid_flag_resetToPeak,base_SdssCentroid_x,base_SdssCentroid_y,base_SdssCentroid_xSigma,base_SdssCentroid_ySigma,base_SdssCentroid_flag,base_SdssCentroid_flag_edge,base_SdssCentroid_flag_noSecondDerivative,base_